# Part 1: Generating data with `FHI-aims` (supporting notebook)

In [3]:
from os.path import exists, join
import chemiscope

import numpy as np
import metatensor as mts
 
from rholearn.utils import cube, system
from rholearn.utils.io import unpickle_dict
from rholearn.options import get_options

dft_options = get_options("dft")
dft_options

{'BASE_AIMS': {'xc': 'pbe0',
  'spin': 'none',
  'charge': 0,
  'relativistic': 'atomic_zora scalar'},
 'SCF': {'elsi_restart': 'write 1000', 'output': 'cube total_density'},
 'RI': {'ri_full_output': True,
  'ri_density_restart': 'write',
  'default_max_l_prodbas': None,
  'default_prodbas_acc': '1e-5',
  'elsi_restart': 'read',
  'ri_skip_scf': True,
  'ri_ovlp_write_triu': True},
 'REBUILD': {'ri_density_restart': 'read 0',
  'ri_full_output': True,
  'default_max_l_prodbas': None,
  'default_prodbas_acc': '1e-5',
  'output': 'cube total_density'},
 'CUBE': {'n_points': [100, 100, 100]},
 'RI_FIT_ID': 'edensity',
 'FIELD_NAME': 'edensity',
 'MASK': None,
 'DATA_DIR': '/home/abbott/codes/rholearn/example/rholearn-aims-tutorial/part-1-dft/data',
 'XYZ': '/home/abbott/codes/rholearn/example/rholearn-aims-tutorial/part-1-dft/data/qm7_16.xyz',
 'AIMS_COMMAND': 'srun /home/abbott/codes/aims_files/binaries/aims.240925.scalapack.mpi.x < control.in > aims.out',
 'SPECIES_DEFAULTS': '/home/ab

In [5]:
# Display the nuclear geometries with chemiscope
frames = system.read_frames_from_xyz(dft_options["XYZ"])
frame_idxs = range(len(frames))
chemiscope.show(system.chemfiles_frame_to_ase_frame(frames), mode="structure")

<StructureWidget(meta={'name': ' '}, structures=[{'size': 21, 'names': ['C', 'C', 'N', 'C', 'N', 'C', 'C', 'H'…

## 1.2: Converge SCF

In [6]:
# Check all SCF calculations have converged
scf_dir = lambda A: f"data/raw/{A}"

not_conv = []
for A in frame_idxs:
    path = join(scf_dir(A), "aims.out")
    if not exists(path):
        not_conv.append(A)
    else:
        with open(path, "r") as f:
            if "Have a nice day." not in f.read():
                not_conv.append(A)

assert len(not_conv) == 0, f"SCF not converged for {not_conv}"

In [7]:
# Confirm again all SCF calculations have converged via the parsed aims.out files
for A in frame_idxs:
    calc_info = unpickle_dict(join(scf_dir(A), "calc_info.pickle"))
    assert calc_info["scf"]["converged"]

In [ ]:
# WARNING: execute with care! 
# Display the electron density volumetric data. This relies on py3Dmol, which is often
# unreliable in jupyter notebooks. Better to use another external software, such as
# VESTA.

# Display the electron density of structure 0
A = 0
rhocube = cube.RhoCube(join(scf_dir(A), "cube_001_total_density.cube"))
rhocube.show_volumetric()

## 1.3: Perform RI decomposition

In [8]:
# Check all RI calculations have finished
ri_dir = lambda A: f"data/raw/{A}/edensity"

not_finished = []
for A in frame_idxs:
    path = join(ri_dir(A), "aims.out")
    if not exists(path):
        not_conv.append(A)
    else:
        with open(path, "r") as f:
            if "Have a nice day." not in f.read():
                not_conv.append(A)

assert len(not_finished) == 0, f"RI not finished for {not_finished}"

In [10]:
# Inpsect the basis set definition for frame 0
processed_dir = lambda A: f"data/processed/{A}/edensity"  # relative dir

A = 0
unpickle_dict(join(processed_dir(A), "basis_set.pickle"))

{'lmax': {'C': 4, 'N': 4, 'H': 2},
 'nmax': {('C', 0): 9,
  ('C', 1): 7,
  ('C', 2): 7,
  ('C', 3): 3,
  ('C', 4): 1,
  ('N', 0): 9,
  ('N', 1): 8,
  ('N', 2): 7,
  ('N', 3): 3,
  ('N', 4): 1,
  ('H', 0): 4,
  ('H', 1): 3,
  ('H', 2): 1}}

In [11]:
# Inspect the metadata structure of the coefficient vector and overlap matrix
coeffs = mts.load(join(processed_dir(A), "ri_coeffs.npz"))
ovlp = mts.load(join(processed_dir(A), "ri_ovlp.npz"))

In [12]:
# The coefficient (and projection) vector is a 1D array, stored in a block sparse
# format. As it represents an equivariant target property (i.e. the density decomposed
# onto a spherical basis), knowing the spherical symmetry of each block - in terms of
# `o3_lambda` and `o3_sigma` - is crucial for unserstanding the bahviour under rotation
# and what equivariance-preserving operations can be applied.
coeffs

TensorMap with 13 blocks
keys: o3_lambda  o3_sigma  center_type
          0         1           6
          1         1           6
                    ...
          3         1           7
          4         1           7

In [13]:
# The first axis of each block is the atomic samples. The intermediate axis is the
# spherical harmonics components, and the properties (last axis) is the radial basis
# function index.
coeffs.block(o3_lambda=1, o3_sigma=1, center_type=6)

TensorBlock
    samples (5): ['system', 'atom']
    components (3): ['o3_mu']
    properties (7): ['radial_n']
    gradients: None

In [14]:
# The overlap matrix is stored with pairs of center types (i.e. chemical species) as
# sparse keys. These have consistent basis set definitions, so overlap matrices between
# atoms can be stacked.
ovlp

TensorMap with 7 blocks
keys: center_1_type  center_2_type
            6              6
            6              7
                  ...
            7              1
            1              1

In [15]:
# For each center type pair, the overlaps between basis functions of pairs of atoms of
# types center_1_type and center_2_type are stacked along the samples (first) axis.
ovlp.block(center_1_type=6, center_2_type=6)

TensorBlock
    samples (15): ['system', 'atom_1', 'atom_2']
    components (95): ['l1_n1_m1']
    properties (95): ['l2_n2_m2']
    gradients: None

In [16]:
# Calculate the normalised MAE, averaged over all structures
nmaes = []
for A in frame_idxs:
    df_error = unpickle_dict(
        join(processed_dir(A), "df_error.pickle")
    )
    nmae = 100 * df_error['abs_error'] / df_error['norm']
    nmaes.append(nmae)

print("NMAE (%)")
print("   Min  : ", np.min(nmaes))
print("   Mean : ", np.mean(nmaes))
print("   Max  : ", np.max(nmaes))

NMAE (%)
   Min  :  0.2397463093683903
   Mean :  0.262958566601245
   Max  :  0.2822949031750377


In [19]:
# Optional: check that the RI rebuilt density output in the RI procesure is equivalent
# to the density rebuilt from the corresponding coefficients.
from rholearn.aims_interface import fields

rebuild_dir = lambda A: f"data/raw/{A}/edensity/rebuild"  # relative dir

# Check for the first structure
A = 0
abs_error, norm = fields.field_absolute_error(
    input=np.loadtxt(join(rebuild_dir(A), "rho_rebuilt_ri.out")),
    target=np.loadtxt(join(ri_dir(A), "rho_rebuilt_ri.out")),
    grid=np.loadtxt(join(ri_dir(A), "partition_tab.out"))
)

assert 100 * abs_error / norm < 1e-6